# Matrix assembly

In [ ]:
from ibvpy.mathkit.linalg.sys_mtx_assembly import SysMtxAssembly
import numpy as np

## Bar 1

In [ ]:
def get_bar_mtx_array(shape):
    '''Get an array of matrices and dof_maps corresponding to
    a bar discretization
    '''
    k_oo = np.array([[10, -10],
                    [-10, 10]], dtype='float_')

    k_Eoo = np.array([k_oo for i in range(shape)], dtype=float)
    O_Eo = np.array([
        np.arange(shape), np.arange(shape) + 1], dtype=int).transpose()

    return O_Eo, k_Eoo

## Index expression for assembly map

In [ ]:
O_Eo, k_Eoo = get_bar_mtx_array(shape=10)

In [ ]:
a = np.arange(2, dtype=np.int_)
O, P = (np.einsum('a,Eo->aEo', (1-a), O_Eo)[:, :, :, None] + 
        np.einsum('a,Ep->aEp', a, O_Eo)[:, :, None, :])

## Example - uniaxial bar

In [ ]:
K = SysMtxAssembly()
K.add_mtx_array(dof_map_arr=O_Eo, mtx_arr=k_Eoo)
K.register_constraint(a=0,  u_a=0.)  # clamped end
K.register_constraint(a=10, u_a=1.)
K.register_constraint(a=10, u_a=1.)
K.toarray()

In [ ]:
R = np.zeros(K.n_dofs)
R

In [ ]:
K.apply_constraints(R)

In [ ]:
K.toarray()

In [ ]:
R

In [ ]:
U_O, _ = K.solve(R)
U_O

## Boundary conditions - linear algebra

$$
\left[
\begin{array}{cc}
K_{ab} & K_{an} \\
K_{mb} & K_{mn} \\
\end{array}
\right]
\left[
\begin{array}{c}
u_{b} \\ u_{n} 
\end{array}
\right]
=
\left[
\begin{array}{c}
P_{a} \\ P_{m} 
\end{array}
\right]
$$

$$
\left[
\begin{array}{c}
u_{b} \\ u_{n} 
\end{array}
\right]
=
\left[
\begin{array}{c}
\alpha_{bp} \\ \delta_{np} 
\end{array}
\right]
u_p
+
\left[
\begin{array}{c}
\delta_{bc} \\ 0_{nc} 
\end{array}
\right]
\bar{u}_{c}
$$

$$
\left[
\begin{array}{cc}
K_{ab} & K_{an} \\
K_{mb} & K_{mn} \\
\end{array}
\right]
\left(
\left[
\begin{array}{c}
\alpha_{bp} \\ \delta_{np} 
\end{array}
\right]
u_p
+
\left[
\begin{array}{c}
\delta_{bc} \\ 0_{nc} 
\end{array}
\right]
\bar{u}_{c}
\right)
=
\left[
\begin{array}{c}
P_{a} \\ P_{m} 
\end{array}
\right]
$$

$$
\left[
\begin{array}{cc}
\alpha_{ao} & \delta_{mo} 
\end{array}
\right]
\left[
\begin{array}{cc}
K_{ab} & K_{an} \\
K_{mb} & K_{mn} \\
\end{array}
\right]
\left(
\left[
\begin{array}{c}
\alpha_{bp} \\ \delta_{np} 
\end{array}
\right]
u_p
+
\left[
\begin{array}{c}
\delta_{bc} \\ 0_{nc} 
\end{array}
\right]
\bar{u}_{c}
\right)
=
\left[
\begin{array}{cc}
\alpha_{ao} & \delta_{mo} 
\end{array}
\right]
\left[
\begin{array}{c}
P_{a} \\ P_{m} 
\end{array}
\right]
$$

$$
\left[
\begin{array}{cc}
\alpha_{ao} & \delta_{mo} 
\end{array}
\right]
\left[
\begin{array}{cc}
K_{ab} & K_{an} \\
K_{mb} & K_{mn} \\
\end{array}
\right]
\left[
\begin{array}{c}
\alpha_{bp} \\ \delta_{np} 
\end{array}
\right]
u_p
=
\left[
\begin{array}{cc}
\alpha_{ao} & \delta_{mo} 
\end{array}
\right]
\left[
\begin{array}{c}
P_{a} \\ P_{m} 
\end{array}
\right]
-
\left[
\begin{array}{cc}
\alpha_{ao} & \delta_{mo} 
\end{array}
\right]
\left[
\begin{array}{cc}
K_{ab} & K_{an} \\
K_{mb} & K_{mn} \\
\end{array}
\right]
\left[
\begin{array}{c}
\delta_{bc} \\ 0_{nc} 
\end{array}
\right]
\bar{u}_{c}
$$

$$
\left[
\begin{array}{cc}
\alpha_{ao} K_{ab} \alpha_{bp} & \alpha_{ao} K_{an} \delta_{np} \\
\delta_{mo} K_{mb} \alpha_{bp} & \delta_{mo} K_{mn} \delta_{np} \\
\end{array}
\right]
u_p
=
\left[
\begin{array}{c}
\alpha_{ao} P_{a} \\ \delta_{mo}  P_{m} 
\end{array}
\right]
-
\left[
\begin{array}{cc}
\alpha_{ao} K_{ab} \delta_{bc} & \alpha_{ao} K_{an} 0_{nc}  \\
\delta_{mo} K_{mb} \delta_{bc} & \delta_{mo} K_{mn} 0_{nc} \\
\end{array}
\right]
\bar{u}_{c}
$$

$$
\left(
\alpha_{ao} K_{ab} \alpha_{bp} + 2 \alpha_{ao} K_{ap} +
K_{op} 
\right)
u_p
=
\alpha_{ao} P_{a} + P_{o} 
-
\left(
\alpha_{ao} K_{ac} + K_{oc} 
\right)
\bar{u}_{c}
$$